In [1]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


###SELECT MODEL TO OPTIMIZE
model_num = 1

5


In [3]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20',
  'model_9_2_20',
  'model_9_10_20'],
 ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'DecisionTreeClassifier(max_depth=96, max_leaf_nodes=573, random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=197,\n                       n_jobs=-1, random_state=75)",
  'GradientBoostingClassifier(max_depth=95, min_samples_leaf=6, n_estimators=195,\n                           random_state=75)',
  'GaussianNB(var_smoothing=1e-12)',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB()',
  'GaussianNB(var_smoothing=1e-12)',
  'GaussianNB(var_smoothing=1e-12)'],
 ['["B_Women\'s Strawweight_rank", \'lose_streak_dif\', \'B_avg_SI

In [4]:
df = pd.read_csv("../data/ufc-master.csv")

In [5]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [6]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [7]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [8]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4208
4208
4208


In [9]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
200,Roosevelt Roberts,Brok Weaver,-360,300,27.777778,300.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Red,...,0.0,0.33,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0
201,Mackenzie Dern,Hannah Cifers,-420,335,23.809524,335.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Red,...,0.0,0.00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
202,Katlyn Chookagian,Antonina Shevchenko,110,-130,110.000000,76.923077,2020-05-30,"Las Vegas, Nevada, USA",USA,Red,...,0.0,1.00,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0
203,Daniel Rodriguez,Gabe Green,-335,275,29.850746,275.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Red,...,0.0,1.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
204,Jamahal Hill,Klidson Abreu,-117,-103,85.470085,97.087379,2020-05-30,"Las Vegas, Nevada, USA",USA,Red,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4403,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,2.0,0.00,0.5,0.0,0.0,0.0,0.0,0.0,0.0,1
4404,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,1.0,1.00,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0
4405,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,2.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4406,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.60,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Holly Holm,Irene Aldana,-125,103,80.000000,103.000000,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Yorgan De Castro,Carlos Felipe,-265,205,37.735849,205.000000,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Germaine de Randamie,Julianna Pena,-150,122,66.666667,122.000000,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,Kyler Phillips,Cameron Else,-480,342,20.833333,342.000000,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,Dequan Townsend,Dusko Todorovic,245,-335,245.000000,29.850746,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Alonzo Menifield,Devin Clark,-225,185,44.444444,185.000000,2020-06-06,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
196,Evan Dunham,Herbert Burns,185,-225,185.000000,44.444444,2020-06-06,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
197,Tyron Woodley,Gilbert Burns,-170,150,58.823529,150.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Blue,...,8.0,0.00,0.25,0.0,1.0,0.0,3.0,0.0,0.0,1
198,Blagoy Ivanov,Augusto Sakai,-105,-115,95.238095,86.956522,2020-05-30,"Las Vegas, Nevada, USA",USA,Blue,...,0.0,0.33,0.00,0.0,0.0,1.0,0.0,0.0,0.0,1


In [10]:
#Set a value for the nulls in the ranks

weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)


D:\Anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [11]:
df_test
#df_test.to_csv('test.csv')

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Holly Holm,Irene Aldana,-125,103,80.000000,103.000000,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Yorgan De Castro,Carlos Felipe,-265,205,37.735849,205.000000,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Germaine de Randamie,Julianna Pena,-150,122,66.666667,122.000000,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,Kyler Phillips,Cameron Else,-480,342,20.833333,342.000000,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,Dequan Townsend,Dusko Todorovic,245,-335,245.000000,29.850746,2020-10-03,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Alonzo Menifield,Devin Clark,-225,185,44.444444,185.000000,2020-06-06,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
196,Evan Dunham,Herbert Burns,185,-225,185.000000,44.444444,2020-06-06,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
197,Tyron Woodley,Gilbert Burns,-170,150,58.823529,150.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Blue,...,8.0,0.00,0.25,0.0,1.0,0.0,3.0,0.0,0.0,1
198,Blagoy Ivanov,Augusto Sakai,-105,-115,95.238095,86.956522,2020-05-30,"Las Vegas, Nevada, USA",USA,Blue,...,0.0,0.33,0.00,0.0,0.0,1.0,0.0,0.0,0.0,1


In [12]:
#1. Set features
#2. Set Hyperparameters
#3. Set EV
#4. Remove Features

In [13]:
models[1][model_num]

'DecisionTreeClassifier(max_depth=96, max_leaf_nodes=573, random_state=75)'

In [14]:
test_model_name = models[0][model_num]
test_model = eval(models[1][model_num])
test_model_features = eval(models[2][model_num])
test_model_ev = eval(models[3][model_num])

In [15]:
old_test_model = test_model
old_test_model_features = test_model_features
old_test_model_ev = test_model_ev

In [16]:
print(df_test.columns.tolist()) #Keeps us from truncating

['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev', 'date', 'location', 'country', 'Winner', 'title_bout', 'weight_class', 'gender', 'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak', 'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT', 'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses', 'B_total_rounds_fought', 'B_total_title_bouts', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms', 'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak', 'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct', 'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought', 'R_total_title_bouts', 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',

In [17]:
#1. set features
my_pos_features = ['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev',
       'location', 'country', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
       'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous',
       'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif',
       'win_streak_dif', 'longest_win_streak_dif', 'win_dif', 'loss_dif',
       'total_round_dif', 'total_title_bout_dif', 'ko_dif', 'sub_dif',
       'height_dif', 'reach_dif', 'age_dif', 'sig_str_dif', 'avg_sub_att_dif',
       'avg_td_dif', 'empty_arena', 'B_match_weightclass_rank', 'R_match_weightclass_rank', 
        "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank",
        "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 
        'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 
        'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", 
        "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 
        'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 
        'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 
        'B_Pound-for-Pound_rank', 'better_rank']

In [18]:
print(test_model_name)
print(test_model)
print(test_model_features)
print(test_model_ev)



model_1
DecisionTreeClassifier(max_depth=96, max_leaf_nodes=573, random_state=75)
["R_Women's Featherweight_rank", 'B_Lightweight_rank', 'B_wins', "B_Women's Flyweight_rank", "B_Women's Strawweight_rank", 'B_Bantamweight_rank', 'R_Reach_cms', 'B_win_by_Decision_Unanimous', 'height_dif', 'B_avg_TD_pct', 'constant_1']
9


In [19]:
def save_model():
    score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)
    models[0][model_num] =  test_model_name 
    models[1][model_num] = test_model
    models[2][model_num] = test_model_features
    models[3][model_num] = test_model_ev
    models[4][model_num] = score    
    
    with open('models.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        for row in models:
            print("HI")
            writer.writerow(row)

    outfile.close()    

In [20]:
def print_model():
    print()
    print(test_model_name)
    print(test_model)
    print(test_model_features)
    print(test_model_ev)
    print()

In [ ]:
keep_going = True
while(keep_going):
    #1. Set Features
    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)
    test_model_features = (get_best_features(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, 
                                             min_ev=test_model_ev))
    print_model()
    save_model()
    #2 Set hyperparameters
    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train, 
                                      min_ev=test_model_ev)
    print_model()
    save_model()
    #3. Set EV
    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)

    old_test_model_features = test_model_features #This prevents
                                                  #an uneccesary loop
    print_model()
    save_model()
    #4. Remove Features
    #def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    test_model_features = remove_to_improve(test_model_features, test_model, df_train, label_train, odds_train, min_ev = test_model_ev)    
    keep_going = False
    if old_test_model != test_model:
        print("The hyperparameters are different")
        print("OLD:")
        print(old_test_model)
        print("NEW:")
        print(test_model)
        keep_going = True
        old_test_model = test_model
    if old_test_model_features != test_model_features:
        print("The features are different")
        print("OLD:")
        print(old_test_model_features)
        print("NEW:")
        print(test_model_features)
        keep_going = True
        old_test_model_features = test_model_features
    if old_test_model_ev != test_model_ev:
        print("The EV is different")
        print("OLD:")
        print(old_test_model_ev)
        print("NEW:")
        print(test_model_ev)
        keep_going = True
        old_test_model_ev = test_model_ev
    print_model()
    save_model()
    

Current best score is: 11397.503775531743
Feature: R_win_by_Decision_Majority Score: 11658.369605044378
Feature: R_win_by_Submission Score: 12417.633600193541
Feature: empty_arena Score: 16041.035858517049
The best feature was empty_arena.  It scored 16041.035858517049
Current best score is: 16041.035858517049
NO IMPROVEMENT
FINAL BEST SCORE: 16041.035858517049

model_1
DecisionTreeClassifier(max_depth=96, max_leaf_nodes=573, random_state=75)
['empty_arena', "R_Women's Featherweight_rank", 'B_Lightweight_rank', 'B_wins', "B_Women's Flyweight_rank", "B_Women's Strawweight_rank", 'B_Bantamweight_rank', 'R_Reach_cms', 'B_win_by_Decision_Unanimous', 'height_dif', 'B_avg_TD_pct', 'constant_1']
9



(3366, 12)

(3366,)

(3366, 2)

(200, 12)

...

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-125,103,0.000000,1.000000,0
1,-265,205,1.000000,0.000000,1
2,-150,122,1.000000,0.000000,0
3,-480,342,0.505495,0.494505,0
4,245,-335,0.728814,0.271186,1
...,...,...,...,...,...
195,-225,185,1.000000,0.000000,1
196,185,-225,0.500000,0.500000,1
197,-170,150,0.000000,1.000000,1
198,-105,-115,1.000000,0.000000,1



          Number of matches: 200
          Number of bets: 125
          Number of winning bets: 47
          Number of losing bets: 78
          Number of underdog bets: 84
          Number of underdog wins: 24
          Number of underdog losses: 60
          Number of Favorite bets: 36
          Number of favorite wins: 20
          Number of favorite losses: 16
          Number of even bets: 5
          Number of even wins: 3
          Number of even losses: 2
          Profit: -1573.655375731631
          Profit per bet: -12.589243005853048
          Profit per match: -7.868276878658155
          
          
HI
HI
HI
HI
HI


Starting New Run for DecisionTree


Previous Best Score: 21557.790367764614
Criterion: gini splitter: random max_depth: 96 min_samples_leaf: 1 max_leaf_nodes: 576 4870.43407361034
Criterion: gini splitter: best max_depth: 96 min_samples_leaf: 1 max_leaf_nodes: 576 21557.790367764614
Criterion: entropy splitter: random max_depth: 96 min_samples_leaf: 1 max_lea

Criterion: entropy splitter: best max_depth: 95 min_samples_leaf: 2 max_leaf_nodes: 575 -1163.7604346122762
Criterion: gini splitter: random max_depth: 97 min_samples_leaf: 2 max_leaf_nodes: 575 2427.3783579803085
Criterion: gini splitter: best max_depth: 97 min_samples_leaf: 2 max_leaf_nodes: 575 10976.979938387893
Criterion: entropy splitter: random max_depth: 97 min_samples_leaf: 2 max_leaf_nodes: 575 2231.6829219704696
Criterion: entropy splitter: best max_depth: 97 min_samples_leaf: 2 max_leaf_nodes: 575 -1163.7604346122762
Criterion: gini splitter: random max_depth: 97 min_samples_leaf: 2 max_leaf_nodes: 575 2427.3783579803085
Criterion: gini splitter: best max_depth: 97 min_samples_leaf: 2 max_leaf_nodes: 575 10976.979938387893
Criterion: entropy splitter: random max_depth: 97 min_samples_leaf: 2 max_leaf_nodes: 575 2231.6829219704696
Criterion: entropy splitter: best max_depth: 97 min_samples_leaf: 2 max_leaf_nodes: 575 -1163.7604346122762
Criterion: gini splitter: random max_d

Criterion: entropy splitter: random max_depth: 95 min_samples_leaf: 1 max_leaf_nodes: 920 -3215.033885134563
Criterion: entropy splitter: best max_depth: 95 min_samples_leaf: 1 max_leaf_nodes: 920 8193.059982394836
Criterion: gini splitter: random max_depth: 97 min_samples_leaf: 1 max_leaf_nodes: 920 11130.683094211217
Criterion: gini splitter: best max_depth: 97 min_samples_leaf: 1 max_leaf_nodes: 920 15496.909510936917
Criterion: entropy splitter: random max_depth: 97 min_samples_leaf: 1 max_leaf_nodes: 920 -3215.033885134563
Criterion: entropy splitter: best max_depth: 97 min_samples_leaf: 1 max_leaf_nodes: 920 8193.059982394836
Criterion: gini splitter: random max_depth: 97 min_samples_leaf: 1 max_leaf_nodes: 920 11130.683094211217
Criterion: gini splitter: best max_depth: 97 min_samples_leaf: 1 max_leaf_nodes: 920 15496.909510936917
Criterion: entropy splitter: random max_depth: 97 min_samples_leaf: 1 max_leaf_nodes: 920 -3215.033885134563
Criterion: entropy splitter: best max_dep

In [ ]:
#Get new score:
#def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels, test_odds, verbose=True):
score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

print(test_model_name) 
print(models[0][model_num])
print()
print(test_model)
print(eval(models[1][model_num]))
print()
print(test_model_features) 
print(eval(models[2][model_num]))
print()
print(test_model_ev)
print(eval(models[3][model_num]))

In [ ]:
models[0][model_num] =  test_model_name 
models[1][model_num] = test_model
models[2][model_num] = test_model_features
models[3][model_num] = test_model_ev
models[4][model_num] = score

In [ ]:
with open('models.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

In [ ]:
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))

In [ ]:
data